In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [3]:
data_path = "/Users/macbook/Downloads/machine_learning_04_data"
train = pd.read_csv(os.path.join(data_path,"train_sample.csv"))
test = pd.read_csv(os.path.join(data_path,"test_sample.csv"))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/macbook/Downloads/machine_learning_04_data/train_sample.csv'

In [ ]:
test.head()

In [ ]:
train.head()

In [5]:
y = train["class"]
X = train.drop("class", axis=1)

In [6]:
param = {'objective': 'binary:logistic', 'eval_metric': 'auc'}

In [1]:
dtrain = xgb.DMatrix(X, label=y)
m = xgb.cv(param, dtrain, nfold = 5,num_boost_round=50)
m.head()

NameError: name 'xgb' is not defined

In [8]:
bestNR = np.argmax(m['test-auc-mean'])
bestAUC = m.loc[bestNR,'test-auc-mean']
print("Best number of rounds = %i; AUC = %f" %(bestNR, bestAUC))

Best number of rounds = 35; AUC = 0.933780


In [9]:
best_xgb = xgb.train(param, dtrain, num_boost_round=bestNR)

In [10]:
prediction = best_xgb.predict(xgb.DMatrix(test))
prediction.shape

(500,)

In [11]:
RF = RandomForestClassifier(random_state=0).fit(X, y)
t = pd.Series(RF.feature_importances_, index=range(1,8))

In [12]:
rf_most_important = t.sort_values(ascending=False).index
rf_most_important

Int64Index([4, 3, 1, 2, 5, 7, 6], dtype='int64')

In [13]:
np.savez('submission.npz', rf_most_important=rf_most_important[0], 
         prediction=prediction)

In [14]:
npzfile = np.load('submission.npz')
print(npzfile['rf_most_important'],npzfile['prediction'].shape)

4 (500,)
